In [2]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import pickle 
import patsy
import seaborn as sns
#from seaborn import plt (alternate)
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_validation import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_selection import f_regression
from sklearn.cross_validation import cross_val_score
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

/home/aglee/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# Read compiled data for regression analysis
X_train = pd.read_pickle('data/X_train.pkl')
X_test = pd.read_pickle('data/X_test.pkl')
y_train = pd.read_pickle('data/y_train.pkl')
y_test = pd.read_pickle('data/y_test.pkl')
fts = pd.read_pickle('data/sign_columns.pkl')
X_trans = pd.read_pickle('data/X_trans.pkl')

In [4]:
# yX = pd.concat([X_trans,y_train])
yX = pd.read_pickle('data/yX_trans.pkl')
# len(yX.columns)
# yX.columns = 
# X = yX.drop('Prices',axis=1)
# y = yX['Prices']
# q('feature')
# Gather features
features = ' + '.join(X_trans.columns)
# features
# print(yX['Prices'])
# y, X = dmatrices('Prices ~ Sq_Ft + District_UnionElementary', data=yX, )
# y, X = dmatrices('Prices ~ Sq_Ft + Years_Since_Built + Days_On_Site + HOA + Fireplace_b + Pool_b + Condominium + SingleFamilyResidence + Townhouse + 0_Cars + 12_Cars + 16_Cars + 1_Cars + 246_Cars + 2_Cars + 35_Cars + 3_Cars + 4_Cars + 5_Cars + 6_Cars + 7_Cars + 8_Cars + 9_Cars + 12_Full_Baths + 1_Full_Baths + 2_Full_Baths + 3_Full_Baths + 4_Full_Baths + 5_Full_Baths + 6_Full_Baths + 7_Full_Baths + 8_Full_Baths + 9_Full_Baths + 0_Part_Baths + 1_Part_Baths + 2_Part_Baths + 3_Part_Baths + 4_Part_Baths + District_AlamedaCityUnified + District_AlamedaUnified + District_AlbanyCityUnified + District_AlumRockUnionElementary + District_BayshoreElementary + District_Belmont_RedwoodShoresElementary + District_BerryessaUnionElementary + District_BrisbaneElementary + District_BurlingameElementary + District_CampbellUnionElementary + District_CastroValleyUnified + District_CupertinoUnion + District_DublinUnified + District_EastSideUnionHigh + District_EmeryUnified + District_EvergreenElementary + District_Franklin_McKinleyElementary + District_FremontUnified + District_GilroyUnified + District_HaywardUnified + District_HillsboroughCityElementary + District_JeffersonElementarySanMateo + District_LasLomitasElementary + District_LivermoreValleyJointUnified + District_LosAltosElementary + District_LosGatosUnionElementary + District_LutherBurbank + District_MenloParkCityElementary + District_MillbraeElementary + District_MilpitasUnified + District_MorelandElementary + District_MorganHillUnified + District_MountainViewWhisman + District_Mt.PleasantElementary + District_NewHavenUnified + District_NewarkUnified + District_OakGroveElementary + District_OaklandUnified + District_OrchardElementary + District_Pacifica + District_PaloAltoUnified + District_PiedmontCityUnified + District_PleasantonUnified + District_PortolaValleyElementary + District_RavenswoodCityElementary + District_RedwoodCityElementary + District_SanBrunoParkElementary + District_SanCarlosElementary + District_SanFranciscoUnified + District_SanJoseUnified + District_SanLeandroUnified + District_SanLorenzoUnified + District_SanMateo_FosterCityElementary + District_SaratogaUnionElementary + District_SouthSanFranciscoUnified + District_Sunnyvale + District_SunolGlenElementary + District_UnionElementary + District_WoodsideElementary', data=yX, return_type='dataframe')
# yX
y, X = dmatrices('Prices ~ ' + features, data=yX, return_type='dataframe')

# y, X = dmatrices('annual_inc ~' + features, df, return_type='dataframe')


In [5]:
features

'SingleFamilyResidence + Cars_0 + Cars_1 + Full_Baths_1 + Full_Baths_3 + Sq_Ft + Years_Since_Built + Days_On_Site + HOA + Fireplace_b + Pool_b + Cars_10 + Cars_12 + Cars_16 + Cars_246 + Cars_35 + Cars_4 + Cars_5 + Cars_6 + Cars_7 + Cars_8 + Cars_9 + Full_Baths_12 + Full_Baths_5 + Full_Baths_6 + Full_Baths_7 + Full_Baths_8 + Full_Baths_9 + Part_Baths_3 + Part_Baths_4 + District_AlamedaCityUnified + District_AlamedaUnified + District_AlbanyCityUnified + District_AlumRockUnionElementary + District_BayshoreElementary + District_Belmont_RedwoodShoresElementary + District_BerryessaUnionElementary + District_BrisbaneElementary + District_BurlingameElementary + District_CabrilloUnified + District_CampbellUnionElementary + District_CastroValleyUnified + District_CupertinoUnion + District_DublinUnified + District_EastSideUnionHigh + District_EmeryUnified + District_EvergreenElementary + District_Franklin_McKinleyElementary + District_FremontUnified + District_GilroyUnified + District_HaywardUnif

In [28]:
vif = pd.DataFrame()
vif["VIFFactor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [44]:
vif.round(1)[(vif.VIFFactor<50)]['features'].unique()

array(['Intercept', 'Sq_Ft', 'Years_Since_Built', 'Days_On_Site', 'HOA',
       'Fireplace_b', 'Pool_b', 'Cars_10', 'Cars_12', 'Cars_16',
       'Cars_246', 'Cars_35', 'Cars_4', 'Cars_5', 'Cars_6', 'Cars_7',
       'Cars_8', 'Cars_9', 'Full_Baths_12', 'Full_Baths_5',
       'Full_Baths_6', 'Full_Baths_7', 'Full_Baths_8', 'Full_Baths_9',
       'Part_Baths_3', 'Part_Baths_4', 'District_AlamedaCityUnified',
       'District_AlamedaUnified', 'District_AlbanyCityUnified',
       'District_AlumRockUnionElementary', 'District_BayshoreElementary',
       'District_Belmont_RedwoodShoresElementary',
       'District_BerryessaUnionElementary', 'District_BrisbaneElementary',
       'District_BurlingameElementary', 'District_CabrilloUnified',
       'District_CampbellUnionElementary', 'District_CastroValleyUnified',
       'District_CupertinoUnion', 'District_DublinUnified',
       'District_EastSideUnionHigh', 'District_EmeryUnified',
       'District_EvergreenElementary',
       'District_Frankl

In [61]:
# vif.round(18)[(vif.VIFFactor>50)]
# vif.round(1)[(vif.VIFFactor>50)]['features'].unique()

,VIFFactor,features
7,inf,Condominium
8,inf,SingleFamilyResidence
9,inf,Townhouse
10,426.135826,Cars_0
11,716.689244,Cars_1
15,1004.154614,Cars_2
17,221.192183,Cars_3
25,659.539853,Full_Baths_1
27,950.761439,Full_Baths_2
28,542.792631,Full_Baths_3


In [56]:
feat = ['SingleFamilyResidence', 'Cars_0',
'Cars_1', 'Cars_2', 'Cars_3', 'Full_Baths_1', 'Full_Baths_2',
'Full_Baths_3', 'Full_Baths_4', 'Sq_Ft', 'Years_Since_Built', 'Days_On_Site', 'HOA',
'Fireplace_b', 'Pool_b', 'Cars_10', 'Cars_12', 'Cars_16',
'Cars_246', 'Cars_35', 'Cars_4', 'Cars_5', 'Cars_6', 'Cars_7',
'Cars_8', 'Cars_9', 'Full_Baths_12', 'Full_Baths_5',
'Full_Baths_6', 'Full_Baths_7', 'Full_Baths_8', 'Full_Baths_9',
'Part_Baths_3', 'Part_Baths_4', 'District_AlamedaCityUnified',
'District_AlamedaUnified', 'District_AlbanyCityUnified',
'District_AlumRockUnionElementary', 'District_BayshoreElementary',
'District_Belmont_RedwoodShoresElementary',
'District_BerryessaUnionElementary', 'District_BrisbaneElementary',
'District_BurlingameElementary', 'District_CabrilloUnified',
'District_CampbellUnionElementary', 'District_CastroValleyUnified',
'District_CupertinoUnion', 'District_DublinUnified',
'District_EastSideUnionHigh', 'District_EmeryUnified',
'District_EvergreenElementary',
'District_Franklin_McKinleyElementary', 'District_FremontUnified',
'District_GilroyUnified', 'District_HaywardUnified',
'District_HillsboroughCityElementary',
'District_JeffersonElementarySanMateo',
'District_LasLomitasElementary',
'District_LivermoreValleyJointUnified',
'District_LosAltosElementary', 'District_LosGatosUnionElementary',
'District_LutherBurbank', 'District_MenloParkCityElementary',
'District_MillbraeElementary', 'District_MilpitasUnified',
'District_MorelandElementary', 'District_MorganHillUnified',
'District_MountainViewWhisman', 'District_MtPleasantElementary',
'District_NewHavenUnified', 'District_NewarkUnified',
'District_OakGroveElementary', 'District_OaklandUnified',
'District_OrchardElementary', 'District_Pacifica',
'District_PaloAltoUnified', 'District_PiedmontCityUnified',
'District_PleasantonUnified', 'District_PortolaValleyElementary',
'District_RavenswoodCityElementary',
'District_RedwoodCityElementary',
'District_SanBrunoParkElementary', 'District_SanCarlosElementary',
'District_SanFranciscoUnified', 'District_SanJoseUnified',
'District_SanLeandroUnified', 'District_SanLorenzoUnified',
'District_SanMateo_FosterCityElementary',
'District_SaratogaUnionElementary',
'District_SouthSanFranciscoUnified', 'District_Sunnyvale',
'District_SunolGlenElementary', 'District_UnionElementary',
'District_WoodsideElementary']
features = ' + '.join(X_trans[feat].columns)
y, X = dmatrices('Prices ~ ' + features, data=yX, return_type='dataframe')

In [62]:
vif = pd.DataFrame()
vif["VIFFactor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [44]:
vif.round(1)[(vif.VIFFactor<50)]['features'].unique()

array(['Intercept', 'Sq_Ft', 'Years_Since_Built', 'Days_On_Site', 'HOA',
       'Fireplace_b', 'Pool_b', 'Cars_10', 'Cars_12', 'Cars_16',
       'Cars_246', 'Cars_35', 'Cars_4', 'Cars_5', 'Cars_6', 'Cars_7',
       'Cars_8', 'Cars_9', 'Full_Baths_12', 'Full_Baths_5',
       'Full_Baths_6', 'Full_Baths_7', 'Full_Baths_8', 'Full_Baths_9',
       'Part_Baths_3', 'Part_Baths_4', 'District_AlamedaCityUnified',
       'District_AlamedaUnified', 'District_AlbanyCityUnified',
       'District_AlumRockUnionElementary', 'District_BayshoreElementary',
       'District_Belmont_RedwoodShoresElementary',
       'District_BerryessaUnionElementary', 'District_BrisbaneElementary',
       'District_BurlingameElementary', 'District_CabrilloUnified',
       'District_CampbellUnionElementary', 'District_CastroValleyUnified',
       'District_CupertinoUnion', 'District_DublinUnified',
       'District_EastSideUnionHigh', 'District_EmeryUnified',
       'District_EvergreenElementary',
       'District_Frankl

In [70]:
# vif.round(78)[(vif.VIFFactor>50)]
vif.round(1)[(vif.VIFFactor>50)]['features'].unique()

array(['Intercept', 'Cars_0', 'Cars_1', 'Cars_2', 'Cars_3',
       'Full_Baths_1', 'Full_Baths_2', 'Full_Baths_3', 'Full_Baths_4'],
      dtype=object)

In [6]:
feat = ['SingleFamilyResidence', 'Cars_0', 'Cars_1', 'Full_Baths_1', 'Full_Baths_3', 'Sq_Ft', 'Years_Since_Built',
'Days_On_Site', 'HOA', 'Fireplace_b', 'Pool_b', 'Cars_10',
'Cars_12', 'Cars_16', 'Cars_246', 'Cars_35', 'Cars_4', 'Cars_5',
'Cars_6', 'Cars_7', 'Cars_8', 'Cars_9', 'Full_Baths_12',
'Full_Baths_5', 'Full_Baths_6', 'Full_Baths_7', 'Full_Baths_8',
'Full_Baths_9', 'Part_Baths_3', 'Part_Baths_4',
'District_AlamedaCityUnified', 'District_AlamedaUnified',
'District_AlbanyCityUnified', 'District_AlumRockUnionElementary',
'District_BayshoreElementary',
'District_Belmont_RedwoodShoresElementary',
'District_BerryessaUnionElementary', 'District_BrisbaneElementary',
'District_BurlingameElementary', 'District_CabrilloUnified',
'District_CampbellUnionElementary', 'District_CastroValleyUnified',
'District_CupertinoUnion', 'District_DublinUnified',
'District_EastSideUnionHigh', 'District_EmeryUnified',
'District_EvergreenElementary',
'District_Franklin_McKinleyElementary', 'District_FremontUnified',
'District_GilroyUnified', 'District_HaywardUnified',
'District_HillsboroughCityElementary',
'District_JeffersonElementarySanMateo',
'District_LasLomitasElementary',
'District_LivermoreValleyJointUnified',
'District_LosAltosElementary', 'District_LosGatosUnionElementary',
'District_LutherBurbank', 'District_MenloParkCityElementary',
'District_MillbraeElementary', 'District_MilpitasUnified',
'District_MorelandElementary', 'District_MorganHillUnified',
'District_MountainViewWhisman', 'District_MtPleasantElementary',
'District_NewHavenUnified', 'District_NewarkUnified',
'District_OakGroveElementary', 'District_OaklandUnified',
'District_OrchardElementary', 'District_Pacifica',
'District_PaloAltoUnified', 'District_PiedmontCityUnified',
'District_PleasantonUnified', 'District_PortolaValleyElementary',
'District_RavenswoodCityElementary',
'District_RedwoodCityElementary',
'District_SanBrunoParkElementary', 'District_SanCarlosElementary',
'District_SanFranciscoUnified', 'District_SanJoseUnified',
'District_SanLeandroUnified', 'District_SanLorenzoUnified',
'District_SanMateo_FosterCityElementary',
'District_SaratogaUnionElementary',
'District_SouthSanFranciscoUnified', 'District_Sunnyvale',
'District_SunolGlenElementary', 'District_UnionElementary',
'District_WoodsideElementary']
features = ' + '.join(X_trans[feat].columns)
y, X = dmatrices('Prices ~ ' + features, data=yX, return_type='dataframe')

In [7]:
vif = pd.DataFrame()
vif["VIFFactor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [8]:
vif.round(1)[(vif.VIFFactor>1)]
# vif.round(1)[(vif.VIFFactor>50)]['features'].unique()

,VIFFactor,features
0,44.0,Intercept
1,3.4,SingleFamilyResidence
2,1.4,Cars_0
3,1.5,Cars_1
4,1.5,Full_Baths_1
5,1.3,Full_Baths_3
6,2.6,Sq_Ft
7,2.1,Years_Since_Built
8,1.0,Days_On_Site
9,2.8,HOA


In [9]:
feat = ['SingleFamilyResidence', 'Cars_0', 'Cars_1', 'Full_Baths_1', 'Full_Baths_3', 'Sq_Ft', 'Years_Since_Built',
'Days_On_Site', 'HOA', 'Fireplace_b', 'Pool_b', 'Cars_10',
'Cars_12', 'Cars_16', 'Cars_246', 'Cars_35', 'Cars_4', 'Cars_5',
'Cars_6', 'Cars_7', 'Cars_8', 'Cars_9', 'Full_Baths_12',
'Full_Baths_5', 'Full_Baths_6', 'Full_Baths_7', 'Full_Baths_8',
'Full_Baths_9', 'Part_Baths_3', 'Part_Baths_4',
'District_AlamedaCityUnified', 'District_AlamedaUnified',
'District_AlbanyCityUnified', 'District_AlumRockUnionElementary',
'District_BayshoreElementary',
'District_Belmont_RedwoodShoresElementary',
'District_BerryessaUnionElementary', 'District_BrisbaneElementary',
'District_BurlingameElementary', 'District_CabrilloUnified',
'District_CampbellUnionElementary', 'District_CastroValleyUnified',
'District_CupertinoUnion', 'District_DublinUnified',
'District_EastSideUnionHigh', 'District_EmeryUnified',
'District_EvergreenElementary',
'District_Franklin_McKinleyElementary', 'District_FremontUnified',
'District_GilroyUnified', 'District_HaywardUnified',
'District_HillsboroughCityElementary',
'District_JeffersonElementarySanMateo',
'District_LasLomitasElementary',
'District_LivermoreValleyJointUnified',
'District_LosAltosElementary', 'District_LosGatosUnionElementary',
'District_LutherBurbank', 'District_MenloParkCityElementary',
'District_MillbraeElementary', 'District_MilpitasUnified',
'District_MorelandElementary', 'District_MorganHillUnified',
'District_MountainViewWhisman', 'District_MtPleasantElementary',
'District_NewHavenUnified', 'District_NewarkUnified',
'District_OakGroveElementary', 'District_OaklandUnified',
'District_OrchardElementary', 'District_Pacifica',
'District_PaloAltoUnified', 'District_PiedmontCityUnified',
'District_PleasantonUnified', 'District_PortolaValleyElementary',
'District_RavenswoodCityElementary',
'District_RedwoodCityElementary',
'District_SanBrunoParkElementary', 'District_SanCarlosElementary',
'District_SanFranciscoUnified', 'District_SanJoseUnified',
'District_SanLeandroUnified', 'District_SanLorenzoUnified',
'District_SanMateo_FosterCityElementary',
'District_SaratogaUnionElementary',
'District_SouthSanFranciscoUnified', 'District_Sunnyvale',
'District_SunolGlenElementary', 'District_UnionElementary',
'District_WoodsideElementary']

In [10]:
len(feat)

90

In [81]:
pd.DataFrame({'features':feat}).to_pickle('data/vif_feat.pkl')
X_trans[feat].to_pickle('data/X_trans.pkl')

In [83]:
yX_columns = ['Prices']
yX_columns = yX_columns+feat
yX = yX[yX_columns]
yX.to_pickle('data/yX_trans.pkl')